In [2]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv() 

openai.api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI()

In [3]:

# respponse = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "What is human life expectancy in the United States?"}
#     ]   
# )   
# print(respponse.choices[0].message.content)

In [4]:
import requests
from io import BytesIO


def create_file(client, file_path):
    with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    # print(result.id)
    return result.id

file_id = create_file(client, "user.txt")

vector_store = client.vector_stores.create(
    name="knowledge_base"
)
print(vector_store.id)

result = client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)
result = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)
print(result)

vs_67d880fa9a64819184f0d214b135acc0
SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-QdxHzqPc6AEWnenh8T4LM6', created_at=1742242043, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_67d880fa9a64819184f0d214b135acc0', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))], has_more=False, object='list', first_id='file-QdxHzqPc6AEWnenh8T4LM6', last_id='file-QdxHzqPc6AEWnenh8T4LM6')


In [6]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="what is the name and rating of the user?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": ["vs_67d747b5c0a48191852b145388d0049d"]
    }]
)
print(response.output[1].content[0].text)

The user's name is **Alex Smith** and his ratings across different platforms are as follows:

- **Codeforces**: 1020
- **Codechef**: 980
- **AtCoder**: 850
- **LeetCode**: Streak of 15 days, Success Rate: 65%.


In [9]:
def get_response(query):
    response = client.responses.create( 
        model="gpt-4o-mini",
        input=query,
        instructions="""
        You're a Competitive Programming Mentor. You're helping a student with a problem. 
        The student is stuck on a problem and asks for help.
        When asked for problem suggestions, suggest problems from these sites:
        Codeforces, Codechef, Atcoder, Leetcode, Hackerrank, Topcoder, and A2OJ Ladder.
        Consider the user's rating, problem difficulty (easy, medium, hard), and problem tags 
        (such as dp, greedy, graph, math, etc.) when suggesting problems.
        - Use the file_search tool to search for the user's file.
        - Use the web_search_preview tool to search for problems. include citations.
        """,
        tools=[
            {
                "type": "file_search",
                "vector_store_ids": ["vs_67d747b5c0a48191852b145388d0049d",],
                # "filters": {
                #     "type": "eq",
                #     "key": "username",
                #     "value": "Alex smith"
                # }
            },
            {
                "type": "web_search_preview"
            }
        ],
        # stream=True
    )
    return response

# Get the streaming response object
stream = get_response("suggest me three problems")

# Iterate through the stream to print out the results
print(stream.output[1].content[0].text)
# import rich
# rich.print(stream)
# print(stream)

Here are three problem suggestions tailored to your strengths and areas for improvement:

1. **Codeforces Problem: A. Watermelon**
   - **Difficulty**: Easy
   - **Tags**: Implementation, Math
   - **Link**: [Problem Link](https://codeforces.com/problemset/problem/4/A)
   - **Description**: Determine if a given watermelon can be divided into two parts such that both parts weigh even numbers.

2. **Codechef Problem: Simple GCD**
   - **Difficulty**: Easy
   - **Tags**: Math
   - **Link**: [Problem Link](https://www.codechef.com/problems/GCD)
   - **Description**: Given two numbers, find their GCD and print it.

3. **LeetCode Problem: Two Sum**
   - **Difficulty**: Easy
   - **Tags**: Array, Hash Table
   - **Link**: [Problem Link](https://leetcode.com/problems/two-sum/)
   - **Description**: Given an array of integers, return indices of the two numbers such that they add up to a specific target.

These problems will help reinforce your problem-solving fundamentals and improve your under

In [ ]:
# import asyncio
# from playwright.async_api import async_playwright

# async def run():
#     async with async_playwright() as p:
#         browser = await p.chromium.launch(
#             headless=False,
#             chromium_sandbox=True,
#             env={},
#             args=[
#                 "--disable-extensions",
#                 "--disable-file-system"
#             ]
#         )
#         page = await browser.new_page()
#         await page.set_viewport_size({"width": 1024, "height": 768})
#         await page.goto("https://codeforces.com")
#         await page.wait_for_timeout(10000)
#         await browser.close()

# # Use await directly without asyncio.run()
# await run()


NotImplementedError: 

In [ ]:
def get_response(query):
    response = client.responses.create( 
        model="computer-use-preview",
        # input=query,
        instructions="""
        You're a Competitive Programming Mentor. You're helping a student with a problem. 
        The student is stuck on a problem and asks for help.
        Wh user's rating, problem difficulty (easy, medium, hard), and problem tags 
        (such as dp, greedy, graph, math, etc.) when suggesting problems.
        - Use the file_search tool to sen asked for problem suggestions, suggest problems from these sites:
        Codeforces, Codechef, Atcoder, Leetcode, Hackerrank, Topcoder, and A2OJ Ladder.
        Consider theearch for the user's file.
        - Use the web_search_preview tool to search for problems. include citations.
        """,
        truncation="auto",
        tools=[
            {
                "type": "computer_use_preview",
                "display_height": 1024,
                "display_width": 768,
                "environment" : "linux"
                
            }
        ],
        
    input=[
        {
            "role": "user",
            "content": query
        }
        # Optional: include a screenshot of the initial state of the environment
        # {
        #     type: "input_image",
        #     image_url: f"data:image/png;base64,{screenshot_base64}"
        # }
    ],
    reasoning={
        "generate_summary": "concise",
    },
        # stream=True
    )
    return response

# Get the streaming response object
stream = get_response("suggest me three problems")

# Iterate through the stream to print out the results
# print(stream.output[1].content[0].text)
print(stream)

Response(id='resp_67d8898cd0648191b59ff13fead6278c0ab370168f67ada9', created_at=1742244236.0, error=None, incomplete_details=None, instructions="\n        You're a Competitive Programming Mentor. You're helping a student with a problem. \n        The student is stuck on a problem and asks for help.\n        When asked for problem suggestions, suggest problems from these sites:\n        Codeforces, Codechef, Atcoder, Leetcode, Hackerrank, Topcoder, and A2OJ Ladder.\n        Consider the user's rating, problem difficulty (easy, medium, hard), and problem tags \n        (such as dp, greedy, graph, math, etc.) when suggesting problems.\n        - Use the file_search tool to search for the user's file.\n        - Use the web_search_preview tool to search for problems. include citations.\n        ", metadata={}, model='computer-use-preview-2025-03-11', object='response', output=[ResponseOutputMessage(id='msg_67d8898d9778819190c322993c5972050ab370168f67ada9', content=[ResponseOutputText(annot